In [1]:
# Add client package to sys path

MLAPI_PATH = '/home/jovyan/mlapi'

import sys

if MLAPI_PATH not in sys.path:
    sys.path.append(MLAPI_PATH)

In [2]:
import requests

In [3]:
%load_ext autoreload
%autoreload 2

# ParquetGetter

In [4]:
from mlapi.celery_task_app.parquet import ParquetGetter
from minio import Minio

In [13]:
# Client args (authentication)
client_args = {
    "endpoint": 'minio:9000',
    "access_key": 'oxxo',
    "secret_key": 'password',
    'secure': False
}
minio_client = Minio(**client_args)

# Dummy request.
data = {
    'name': 'sample',
    'algorithm': 'seq2seq',
    'forecast_horizon': 10,
    'perform_hpo': False,
    'bucket_name': 'sample'
}
predictor = Forecaster(**data)


# Parquet getter: getter for parquet datasets stored in minio buckets.
parquet_getter = ParquetGetter(
    'sample', 
    minio_client=minio_client
)

In [ ]:
minio_client.

# Forecasting task

In [4]:
from mlapi.celery_task_app.forecasting import PreprocessorCreator
from mlapi.celery_task_app.forecasting import EstimatorCreator
from mlapi.celery_task_app.forecasting import TimeSeriesMerger
from mlapi.models import Forecaster

In [9]:
# Create data merger.
resolved = parquet_getter.resolve_datasets()
merger = TimeSeriesMerger(**resolved)

# Create preprocessor.
group_ids = merger.get_group_ids()
preprocessor = PreprocessorCreator(group_ids).create_preprocessor()

# Create estimator
estimator = EstimatorCreator(predictor, merger).create_estimator()

In [10]:
X = merger.merge()

In [11]:
preprocessor.fit_transform(X)

,target,group_id_0,group_id_1,timestamp
0,0.105960,10,13,2013-01-01
1,0.178808,10,13,2013-01-02
2,0.039735,10,13,2013-01-03
3,0.158940,10,13,2013-01-04
4,0.165563,10,13,2013-01-05
...,...,...,...,...
91295,0.220238,9,28,2017-12-27
91296,0.303571,9,28,2017-12-28
91297,0.404762,9,28,2017-12-29
91298,0.380952,9,28,2017-12-30
